# Overview

In this notebook, I use LDA to model latent topics. Latent models don't always yield useful results and it is difficult to know whether they will be useful or not until we actually start. 

Steps:

1. Group the Data by restauarant and then squish the review text together. 

Extensions:

1. Consider only positive reviews. (Happy in the same say, upset for all different ways)
2. COnsider only negative reviews. 

In [1]:
import pandas as pd
from gensim.models.ldamulticore import LdaMulticore
import gensim

In [2]:
combined_data = pd.read_hdf('../data/restaurant_reviews.hdf')

In [4]:
combined_data.columns

Index(['date', 'review_id', 'text', 'user_id', 'city', 'latitude', 'longitude',
       'name', 'neighborhoods', 'stars', 'hours'],
      dtype='object')

In [3]:
g_rest = combined_data.groupby(combined_data.name)

In [4]:
len(g_rest.stars.mean())

15321

In [5]:
text = g_rest.text.apply(lambda x: '\n'.join(x)).reset_index()

In [6]:
text.text

0        So this place isn't spectacular, but it is def...
1        I really like this place. The portions are lar...
2        Delicious Sushi! Very big, fresh, tasty, flavo...
3        Pizza was just plain, nothing special. I have ...
4        I don't like the pizza hear and if you go for ...
5        I used to hang out here alot in my high school...
6        For the best Chinese buffet dining in Eastern ...
7        As far as listing what's good about this place...
8        Drive through mexican food is truly 100% Natur...
9        I really wanted to like this place, given the ...
10       Brand new 101 here hit the ground running this...
11       First of all, please forgive. I have been AWOL...
12       I have the number for this take away programme...
13       Nice place in the heart of Wesmount. \n\nI wen...
14       Hubby and I had been excited a few weeks ago w...
15       God I so wish I liked this place. The inside i...
16       This is only getting 3 stars because it is in .

## Basic Preprocessing

In [7]:
text.text = text.text.apply(lambda x: gensim.utils.simple_preprocess(x))

In [8]:
text.text

0        [so, this, place, isn, spectacular, but, it, i...
1        [really, like, this, place, the, portions, are...
2        [delicious, sushi, very, big, fresh, tasty, fl...
3        [pizza, was, just, plain, nothing, special, ha...
4        [don, like, the, pizza, hear, and, if, you, go...
5        [used, to, hang, out, here, alot, in, my, high...
6        [for, the, best, chinese, buffet, dining, in, ...
7        [as, far, as, listing, what, good, about, this...
8        [drive, through, mexican, food, is, truly, nat...
9        [really, wanted, to, like, this, place, given,...
10       [brand, new, here, hit, the, ground, running, ...
11       [first, of, all, please, forgive, have, been, ...
12       [have, the, number, for, this, take, away, pro...
13       [nice, place, in, the, heart, of, wesmount, we...
14       [hubby, and, had, been, excited, few, weeks, a...
15       [god, so, wish, liked, this, place, the, insid...
16       [this, is, only, getting, stars, because, it, .

In [9]:
# import spacy
# import os
# data_dir = '/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/spacy/en/data'
# nlp = spacy.en.English(data_dir=data_dir)

Now we need to make a gensim dictionary that 

In [12]:
dictionary = gensim.corpora.Dictionary(text.text)

In [14]:
dictionary.save('../data/words.dict') 

In [16]:
print(dictionary)

Dictionary(242663 unique tokens: ['grubbiest', 'yearning', 'cockers', 'guus', 'rammelvoll']...)


Now we get the Corpus into the format we want and save it to disk.

In [25]:
corpus = [dictionary.doc2bow(t) for t in text.text]

In [27]:
from gensim import corpora
corpora.MmCorpus.serialize('../data/deerwester.mm', corpus)

In [ ]:
all_lda = LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=15)

# Positive Reviews Only

In [35]:
good_reviews = combined_data[combined_data.stars.isin(['4,5'])]

## Negative Reviews Only

# References

[Good tutorial](https://radimrehurek.com/gensim/tut1.html#corpus-formats) on how to get Corpora in the right format.  

http://nbviewer.ipython.org/gist/langmore/6820351